In [18]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error 
from sklearn.ensemble import IsolationForest 
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt 
import numpy as np
from xgboost import XGBRegressor
from boruta import BorutaPy

In [3]:
acs = pd.read_csv(r'C:\Users\IknaShillingford\Downloads\acs5yr2010_cook.csv') 
mapping = pd.read_excel('C:\\Users\\IknaShillingford\\Downloads\\il17trf.xlsx', sheet_name='il17trf')
energy = pd.read_csv(r'C:\Users\IknaShillingford\Downloads\energy_chicago.csv')

In [4]:
acs['GEO_ID'] = acs.GEO_ID.str.rsplit('US',expand=True)[1]
acs = pd.merge(acs, mapping[['GEOID10','AREALANDPT']], how='left', left_on= acs['GEO_ID'], right_on= mapping['GEOID10'].astype('str'))
acs['GEOID10'].isna().sum()

energy['GEO_ID_TRACT'] = energy['CENSUS BLOCK'].astype('str').str[:11]
e_tot = energy.groupby(['COMMUNITY AREA NAME','GEO_ID_TRACT'])[['TOTAL KWH','TOTAL THERMS']].sum()

df = acs.merge(e_tot, how='right', right_on='GEO_ID_TRACT', left_on='GEO_ID')
df['GEOID10'].isna().sum()
df.drop(columns=['key_0','Unnamed: 0'], inplace=True)
df.to_csv('acs_energy.csv')
df = df.dropna()


y1 = df['TOTAL KWH']
y2 = df['TOTAL THERMS']
drop_cols = [0,1,2,3,4,-1,-2,-4]
X = df.drop(df.columns[drop_cols], axis=1)

In [5]:
missing_values = X.isna().sum()
columns_with_missing_values = missing_values[missing_values > 0]

# Print the columns with missing values along with the number of missing values
print("Columns with missing values:")
print(columns_with_missing_values)

params = dict()
params["device"] = "cuda"
np.int = np.int32
np.float = np.float64
np.bool = np.bool_


X.reset_index(drop=True, inplace=True)

# Check for missing values
print("\nMissing Values:")
print(X.isnull().sum())

# Get summary information about the DataFrame
print("\nDataFrame Info:")
print(X.info())


Columns with missing values:
Series([], dtype: int64)

Missing Values:
Estimate!!BEDROOMS!!1 bedroom                          0
Estimate!!BEDROOMS!!2 bedrooms                         0
Estimate!!BEDROOMS!!3 bedrooms                         0
Estimate!!BEDROOMS!!4 bedrooms                         0
Estimate!!BEDROOMS!!5 or more bedrooms                 0
                                                      ..
Estimate!!YEAR STRUCTURE BUILT!!Built 1990 to 1999     0
Estimate!!YEAR STRUCTURE BUILT!!Built 2000 to 2004     0
Estimate!!YEAR STRUCTURE BUILT!!Built 2005 or later    0
Estimate!!YEAR STRUCTURE BUILT!!Total housing units    0
AREALANDPT                                             0
Length: 218, dtype: int64

DataFrame Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1015 entries, 0 to 1014
Columns: 218 entries, Estimate!!BEDROOMS!!1 bedroom to AREALANDPT
dtypes: float64(19), int64(199)
memory usage: 1.7 MB
None


In [6]:
X.reset_index(drop=True, inplace=True)
y1.reset_index(drop=True, inplace=True)
y2.reset_index(drop=True, inplace=True)

# Replace the '!' characters in column names with underscores
X.columns = X.columns.str.replace('!', '_')
X.columns = X.columns.str.replace('Estimate__', '')
#X.columns = X.columns.str.replace(' ', '')
# Check the modified column names
print(X.columns)

X.reset_index(drop=True, inplace=True)


#Feature selection based on electricity consumption as target variable
xgb = XGBRegressor(n_estimators=100, random_state=6, params=params)
boruta_selector = BorutaPy(xgb, n_estimators='auto', verbose=2, random_state=6, perc=85)
boruta_selector.fit(X.to_numpy(), y2)

Index(['BEDROOMS__1 bedroom', 'BEDROOMS__2 bedrooms', 'BEDROOMS__3 bedrooms',
       'BEDROOMS__4 bedrooms', 'BEDROOMS__5 or more bedrooms',
       'BEDROOMS__No bedroom', 'BEDROOMS__Total housing units',
       'CLASS OF WORKER__Civilian employed population 16 years and over',
       'CLASS OF WORKER__Government workers',
       'CLASS OF WORKER__Private wage and salary workers',
       ...
       'YEAR STRUCTURE BUILT__Built 1940 to 1949',
       'YEAR STRUCTURE BUILT__Built 1950 to 1959',
       'YEAR STRUCTURE BUILT__Built 1960 to 1969',
       'YEAR STRUCTURE BUILT__Built 1970 to 1979',
       'YEAR STRUCTURE BUILT__Built 1980 to 1989',
       'YEAR STRUCTURE BUILT__Built 1990 to 1999',
       'YEAR STRUCTURE BUILT__Built 2000 to 2004',
       'YEAR STRUCTURE BUILT__Built 2005 or later',
       'YEAR STRUCTURE BUILT__Total housing units', 'AREALANDPT'],
      dtype='object', length=218)


c:\Users\IknaShillingford\AppData\Local\Programs\Python\Python311\Lib\site-packages\xgboost\core.py:160: UserWarning: [18:20:29] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0b3782d1791676daf-1\xgboost\xgboost-ci-windows\src\learner.cc:742: 
Parameters: { "params" } are not used.

  warnings.warn(smsg, UserWarning)


Iteration: 	1 / 100
Confirmed: 	0
Tentative: 	218
Rejected: 	0


c:\Users\IknaShillingford\AppData\Local\Programs\Python\Python311\Lib\site-packages\xgboost\core.py:160: UserWarning: [18:20:39] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0b3782d1791676daf-1\xgboost\xgboost-ci-windows\src\learner.cc:742: 
Parameters: { "params" } are not used.

  warnings.warn(smsg, UserWarning)


Iteration: 	2 / 100
Confirmed: 	0
Tentative: 	218
Rejected: 	0


...

c:\Users\IknaShillingford\AppData\Local\Programs\Python\Python311\Lib\site-packages\xgboost\core.py:160: UserWarning: [18:27:34] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0b3782d1791676daf-1\xgboost\xgboost-ci-windows\src\learner.cc:742: 
Parameters: { "params" } are not used.

  warnings.warn(smsg, UserWarning)


Iteration: 	98 / 100
Confirmed: 	48
Tentative: 	23
Rejected: 	147


c:\Users\IknaShillingford\AppData\Local\Programs\Python\Python311\Lib\site-packages\xgboost\core.py:160: UserWarning: [18:27:36] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0b3782d1791676daf-1\xgboost\xgboost-ci-windows\src\learner.cc:742: 
Parameters: { "params" } are not used.

  warnings.warn(smsg, UserWarning)


Iteration: 	99 / 100
Confirmed: 	48
Tentative: 	23
Rejected: 	147


BorutaPy finished running.

Iteration: 	100 / 100
Confirmed: 	48
Tentative: 	3
Rejected: 	147


BorutaPy(estimator=XGBRegressor(base_score=None, booster=None, callbacks=None,
                                colsample_bylevel=None, colsample_bynode=None,
                                colsample_bytree=None, device=None,
                                early_stopping_rounds=None,
                                enable_categorical=False, eval_metric=None,
                                feature_types=None, gamma=None,
                                grow_policy=None, importance_type=None,
                                interaction_constraints=None,
                                learning_rate=None, max_bin=None,
                                max_cat_threshold=None, max_cat_to_onehot=None,
                                max_delta_step=None, max_depth=None,
                                max_leaves=None, min_child_weight=None,
                                missing=nan, monotone_constraints=None,
                                multi_strategy=None, n_estimators=119,
                                n_jobs=None, num_parallel_tree=None,
                                params={'device': 'cuda'}, ...),
         n_estimators='auto', perc=85,
         random_state=RandomState(MT19937) at 0x2A4DC7CD840, verbose=2)

In [7]:
#Output feature selection results
feature_names = X.columns
feature_ranks = list(zip(feature_names, boruta_selector.ranking_, boruta_selector.support_, boruta_selector.support_weak_ )) 
boruta_results_1 = pd.DataFrame(feature_ranks, columns=['feature_name','rank', 'confirmed','tentative'])

#Feature selection based on electricity consumption as target variable
xgb = XGBRegressor(n_estimators=100, random_state=6, params=params)
boruta_selector = BorutaPy(xgb, n_estimators='auto', verbose=2, random_state=6, perc=85)
boruta_selector.fit(X.to_numpy(), y1)

#output feature selection results (electricity as target variable)
feature_names = X.columns
feature_ranks = list(zip(feature_names, boruta_selector.ranking_, boruta_selector.support_, boruta_selector.support_weak_ )) 
boruta_results_2 = pd.DataFrame(feature_ranks, columns=['feature_name','rank', 'confirmed','tentative'])

c:\Users\IknaShillingford\AppData\Local\Programs\Python\Python311\Lib\site-packages\xgboost\core.py:160: UserWarning: [18:27:39] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0b3782d1791676daf-1\xgboost\xgboost-ci-windows\src\learner.cc:742: 
Parameters: { "params" } are not used.

  warnings.warn(smsg, UserWarning)


Iteration: 	1 / 100
Confirmed: 	0
Tentative: 	218
Rejected: 	0


c:\Users\IknaShillingford\AppData\Local\Programs\Python\Python311\Lib\site-packages\xgboost\core.py:160: UserWarning: [18:27:50] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0b3782d1791676daf-1\xgboost\xgboost-ci-windows\src\learner.cc:742: 
Parameters: { "params" } are not used.

  warnings.warn(smsg, UserWarning)


Iteration: 	2 / 100
Confirmed: 	0
Tentative: 	218
Rejected: 	0


c:\Users\IknaShillingford\AppData\Local\Programs\Python\Python311\Lib\site-packages\xgboost\core.py:160: UserWarning: [18:28:02] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0b3782d1791676daf-1\xgboost\xgboost-ci-windows\src\learner.cc:742: 
Parameters: { "params" } are not used.

  warnings.warn(smsg, UserWarning)


...

Iteration: 	97 / 100
Confirmed: 	47
Tentative: 	27
Rejected: 	144


c:\Users\IknaShillingford\AppData\Local\Programs\Python\Python311\Lib\site-packages\xgboost\core.py:160: UserWarning: [18:35:13] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0b3782d1791676daf-1\xgboost\xgboost-ci-windows\src\learner.cc:742: 
Parameters: { "params" } are not used.

  warnings.warn(smsg, UserWarning)


Iteration: 	98 / 100
Confirmed: 	48
Tentative: 	26
Rejected: 	144


c:\Users\IknaShillingford\AppData\Local\Programs\Python\Python311\Lib\site-packages\xgboost\core.py:160: UserWarning: [18:35:20] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0b3782d1791676daf-1\xgboost\xgboost-ci-windows\src\learner.cc:742: 
Parameters: { "params" } are not used.

  warnings.warn(smsg, UserWarning)


Iteration: 	99 / 100
Confirmed: 	48
Tentative: 	26
Rejected: 	144


BorutaPy finished running.

Iteration: 	100 / 100
Confirmed: 	48
Tentative: 	9
Rejected: 	144


In [8]:
# Get the features to keep based on Boruta results
features_to_keep = boruta_results_1[(boruta_results_1['confirmed'] == True) | (boruta_results_1['tentative'] == True)]['feature_name'].tolist()
# Filter the DataFrame to keep only the selected features
df_filtered = X[features_to_keep]

# Now 'df_filtered' contains only the features selected by Boruta (confirmed or tentative)
df_filtered['TOTAL KWH'] = y1 
#df_filtered['TOTAL THERMS'] = y2 


C:\Users\IknaShillingford\AppData\Local\Temp\ipykernel_43512\905488634.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered['TOTAL KWH'] = y1


In [9]:
df_filtered.to_csv('output.csv', index=False)

In [10]:
from sklearn.preprocessing import StandardScaler

In [11]:
# Separate features and target variable
Xs = df_filtered.drop(['TOTAL KWH'], axis=1)  # Assuming 'TOTAL KWH' is the target variable
ys = df_filtered['TOTAL KWH']
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(Xs, ys, test_size=0.2, random_state=42)

# Define a dictionary to store MSE for each model
mse_dict = {}

In [12]:
# Train and evaluate Linear Regression
lr_model = LinearRegression()
lr_model.fit(X_train, y_train)
y_pred_lr = lr_model.predict(X_test)
mse_lr = mean_squared_error(y_test, y_pred_lr)
mse_dict['Linear Regression'] = mse_lr

In [29]:
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import GradientBoostingRegressor  
from sklearn.ensemble import RandomForestRegressor 
from xgboost import XGBRegressor

In [22]:
# Train and evaluate Ridge Regression
ridge_model = Ridge(alpha=1.0)
ridge_model.fit(X_train, y_train)
y_pred_ridge = ridge_model.predict(X_test)
mse_ridge = mean_squared_error(y_test, y_pred_ridge)
mse_dict['Ridge Regression'] = mse_ridge


In [23]:

# Train and evaluate Lasso Regression
lasso_model = Lasso(alpha=1.0)
lasso_model.fit(X_train, y_train)
y_pred_lasso = lasso_model.predict(X_test)
mse_lasso = mean_squared_error(y_test, y_pred_lasso)
mse_dict['Lasso Regression'] = mse_lasso

c:\Users\IknaShillingford\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:678: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.711e+16, tolerance: 8.866e+12
  model = cd_fast.enet_coordinate_descent(


In [24]:
# Train and evaluate Decision Tree
tree_model = DecisionTreeRegressor(random_state=42)
tree_model.fit(X_train, y_train)
y_pred_tree = tree_model.predict(X_test)
mse_tree = mean_squared_error(y_test, y_pred_tree)
mse_dict['Decision Tree'] = mse_tree


In [27]:
# Train and evaluate Random Forest
forest_model = RandomForestRegressor(n_estimators=100, random_state=42)
forest_model.fit(X_train, y_train)
y_pred_forest = forest_model.predict(X_test)
mse_forest = mean_squared_error(y_test, y_pred_forest)
mse_dict['Random Forest'] = mse_forest

In [72]:
# Train and evaluate Gradient Boosting
gb_model = GradientBoostingRegressor(n_estimators=100, learning_rate=0.1, random_state=42)
gb_model.fit(X_train, y_train)
y_pred_gb = gb_model.predict(X_test)
mse_gb = mean_squared_error(y_test, y_pred_gb)
mse_dict['Gradient Boosting'] = mse_gb


In [31]:
# Train and evaluate XGBoost
xgb_model = XGBRegressor(objective='reg:squarederror', random_state=42)
xgb_model.fit(X_train, y_train)
y_pred_xgb = xgb_model.predict(X_test)
mse_xgb = mean_squared_error(y_test, y_pred_xgb)
mse_dict['XGBoost'] = mse_xgb

In [32]:
best_model = min(mse_dict, key=mse_dict.get)
best_mse = mse_dict[best_model]
print(f'Best Model: {best_model}')
print(f'Best Mean Squared Error: {best_mse}')


Best Model: XGBoost
Best Mean Squared Error: 26328014568543.56


In [35]:
from sklearn.model_selection import GridSearchCV

# Define XGBoost regressor model
xgb_model = XGBRegressor(objective='reg:squarederror', random_state=42)

# Define hyperparameters for grid search
param_grid = {
    'learning_rate': [0.05, 0.1, 0.2],
    'max_depth': [3, 5, 7],
    'min_child_weight': [1, 3, 5],
    'gamma': [0, 0.1, 0.2],
    'subsample': [0.6, 0.8, 1.0],
    'colsample_bytree': [0.6, 0.8, 1.0]
}

# Perform grid search for hyperparameter tuning
grid_search = GridSearchCV(estimator=xgb_model, param_grid=param_grid, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)
grid_search.fit(X_train, y_train)

# Get the best estimator from grid search
best_xgb_model = grid_search.best_estimator_

# Predict energy usage for test data using the best model
y_pred_best_xgb = best_xgb_model.predict(X_test)

# Evaluate the best model
mse_best_xgb = mean_squared_error(y_test, y_pred_best_xgb)
print('Mean Squared Error (Best XGBoost after Hyperparameter Tuning):', mse_best_xgb)

# Print the best hyperparameters
print('Best Hyperparameters:', grid_search.best_params_)


Mean Squared Error (Best XGBoost after Hyperparameter Tuning): 60181118237466.34
Best Hyperparameters: {'colsample_bytree': 0.6, 'gamma': 0, 'learning_rate': 0.2, 'max_depth': 5, 'min_child_weight': 1, 'subsample': 1.0}


In [36]:
# Save the best model to a file
import joblib
joblib.dump(best_xgb_model, 'best_xgb_model.pkl')

# Load the model from the file
#loaded_model = joblib.load('best_xgb_model.pkl')

# Use the loaded model to make predictions
#predictions = loaded_model.predict(new_data)

['best_xgb_model.pkl']